# Valors realistes
En aquest script modificaré el codi "Fast & slow modes.ipynb" i definir les variables que entren en joc en les equacions de manera que els inputs siguin variables TD i constants universals.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3
from scipy.optimize import fsolve
import logging
logger = logging.getLogger(__name__)
import scienceplots
plt.style.use('science')



""" Funcions per calcular els autovalors exactes """
# Funcions den Monxo per calcular els autovalors exactes d'una corda amb densitat variable.
def dispersion_relation(omega):
    disprel_even = (np.sqrt(cp2)/np.sqrt(cc2)) * np.cos(omega * (xmax - xp) / np.sqrt(cc2)) * np.cos(omega * xp / np.sqrt(cp2)) - \
        np.sin(omega * (xmax - xp) / np.sqrt(cc2)) * np.sin(omega * xp / np.sqrt(cp2))
    disprel_odd = (np.sqrt(cp2)/np.sqrt(cc2)) * np.cos(omega * (xmax - xp) / np.sqrt(cc2)) * np.sin(omega * xp / np.sqrt(cp2)) + \
        np.sin(omega * (xmax - xp) / np.sqrt(cc2)) * np.cos(omega * xp / np.sqrt(cp2))
    disprel = disprel_even * disprel_odd
    return disprel

def dispersion_relation2(omega):
    disprel_even2 = (np.sqrt(vAp2)/np.sqrt(vAc2)) * np.cos(omega * (xmax - xp) / np.sqrt(vAc2)) * np.cos(omega * xp / np.sqrt(vAp2)) - \
        np.sin(omega * (xmax - xp) / np.sqrt(vAc2)) * np.sin(omega * xp / np.sqrt(vAp2))
    disprel_odd2 = (np.sqrt(vAp2)/np.sqrt(vAc2)) * np.cos(omega * (xmax - xp) / np.sqrt(vAc2)) * np.sin(omega * xp / np.sqrt(vAp2)) + \
        np.sin(omega * (xmax - xp) / np.sqrt(vAc2)) * np.cos(omega * xp / np.sqrt(vAp2))
    disprel2 = disprel_even2 * disprel_odd2
    return disprel2

def get_eigenvalues(Nevals):
    Nsol = 0
    omega0 = 1
    omega_step = 0.5
    exact_soln = np.array([])
    while Nsol < Nevals:
        sol = fsolve(dispersion_relation, omega0)
#         print('omega0, sol ', omega0, sol)
        sol2 = fsolve(dispersion_relation2, omega0)

        if sol > 0:
            exact_soln = np.append(exact_soln, sol)
            exact_soln = np.unique(exact_soln.round(decimals=8))
#         print('exact_soln, exact_soln**2 ', exact_soln, exact_soln**2)
#         print(exact_soln.size)
        if sol2 > 0:
            exact_soln = np.append(exact_soln, sol2)
            exact_soln = np.unique(exact_soln.round(decimals=8))

        Nsol = exact_soln.size
        omega0 = omega0 + omega_step
#    print('exact_soln ', exact_soln[0:5])
#    print('exact_soln**2 ', exact_soln**2)
    print()
    return exact_soln

""" Exact parameters """
def MHD_params(T0p, T0c, rho0p, B0, multildep, multildec):
    # Physical constants in SI units.
    kB = 1.38e-23
    mp = 1.67e-27
    capR = kB / mp
    mu = np.pi * 4e-7
    gamma = 5/3

    # Parameter values in SI units.
    #B0 = 5e-4 # encara que no surti directament a ses equacions el feim servir per fer calcular altres constants.
    #T0p = 8e3
    #multildep = 0.8
    #rho0p = 5e-11
    #T0c = 1e6
    #multildec = 0.5

    p0 = rho0p * capR * T0p / multildep
    rho0c = p0 / (capR * T0c / multildec)
    print('p0 ', p0)
    print('rho0c ', rho0c)
    rho0c = rho0p * T0p / multildep / (T0c / multildec)
    print("rho0c ", rho0c)
    # Sound and Alfvén speeds.
    csp = np.sqrt(gamma * capR * T0p / multildep)
    # csp = np.sqrt(gamma * p0 / rho0p)
    vAp = B0 / np.sqrt(mu * rho0p)
    csc = np.sqrt(gamma * capR * T0c / multildec)
    # csc = np.sqrt(gamma * p0 / rho0c)
    vAc = B0 / np.sqrt(mu * rho0c)
    return csp, csc, vAp, vAc

csp, csc, vAp, vAc = MHD_params(T0p=8e3, T0c=1e6, rho0p=5e-11, B0=5e-4, multildep=0.8, multildec=0.5)
cp2 = csp * csp # 1
cc2 = csc * csc # 6
vAp2 = vAp * vAp # 10
vAc2 = vAc * vAc # 36

print('csp, csc, vAp, vAc ', csp, csc, vAp, vAc)
print('cp2, cc2, vAp2, vAc2 ', cp2, cc2, vAp2, vAc2)



""" Paràmetres """
Nx = 128 # amb 32 ja no és smooth...
dtype = np.complex128 # malla en la que es treballa: 128 complexes, ja que hem definit Nx = 128



""" Límits del domini """
xmax = 1
xmin = -1 # = -xmax
# Position of boundaries between different parts of the system (prominence and corona).
xp = 0.1
# xc ∈ [-1, -0.1], xc ∈ [0.1, 1]



""" Constants """
kz = 0.01
#B0 = 0

### Velocitats a trossos ###
#cp2 = csp * csp # 1
#cc2 = csc * csc # 6
#vAp2 = vAp * vAp # 10
#vAc2 = vAc * vAc # 36



""" Bases """
xcoord = d3.Coordinate('x')
dist = d3.Distributor(xcoord, dtype=dtype)
xbasis = d3.Legendre(xcoord, size=Nx, bounds=(xmin, xmax)) # Definim una base amb uns altres límits. Chebyshev menos error?



""" Camps """
### Velocitats ###
u = dist.Field(name='u', bases=xbasis) # u = field of the wave function - AUTOFUNCIÓ!!! - v10x bar, coplexa
w = dist.Field(name='w', bases=xbasis) # w = Velocity in the x direction   v10z - real
# Definesc la velocitat com en oceano: (u,v,w) per no liar subíndexos i derivades.

### Espai real ###
x = dist.local_grid(xbasis)

### Constants ###
c2 = dist.Field(name='c2', bases=xbasis) # c2 = Velocitat al quadrat ----- és interessant definir-la directament al quadrat ja que és un camp i així evitam fer c.evaluate()**2
c2['g'] = np.piecewise(x, [x < -xp, (-xp <= x) * (x <= xp), x > xp], [cc2, cp2, cc2]) # Representa la c^2 en l'espai real ('g') a trossos.
vA2 = dist.Field(name='vA2', bases=xbasis) # vA2 = Alfven velocity squared
vA2['g'] = np.piecewise(x, [x < -xp, (-xp <= x) * (x <= xp), x > xp], [vAc2, vAp2, vAc2]) # Representa la velocitat d'Alfven en l'espai real ('g') a trossos.

### Autovalors ###
s = dist.Field(name='s') # AUTOVALOR ω^2 = s

### Camps magnètics ###
#Bx = dist.Field(name='Bx', bases=xbasis) # Bx = Magnetic field in the x direction   B10x
#Bz = dist.Field(name='Bz', bases=xbasis) # Bz = Magnetic field in the z direction   B10z

### Densitats ###
#rho0 = dist.Field(name='rho0', bases=xbasis) # rho0 = Density of the plasma  ρ0
#rho10 = dist.Field(name='rho10', bases=xbasis) # rho = Variation density of the plasma   ρ10
#rho10['g'] = np.piecewise(x, [x < -xp, (-xp <= x) * (x <= xp), x > xp], [1, 1, 1]) # Representa la densitat en l'espai real ('g') a trossos.

### Auxiliars de les derivades ###
tau_1 = dist.Field(name='tau_1')
tau_2 = dist.Field(name='tau_2')
tau_3 = dist.Field(name='tau_3')
tau_4 = dist.Field(name='tau_4')



""" Derivatives """
### Derivades espacials ###
dx = lambda A: d3.Differentiate(A, xcoord) # Derivative operator
lift_basis = xbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)
ux = dx(u) + lift(tau_1) # First-order reduction of the velocity in the x direction
uxx = dx(ux) + lift(tau_2) # Second-order reduction of the velocity in the x direction
wx = dx(w) + lift(tau_3) # First-order reduction of the velocity in the z direction
wxx = dx(wx) + lift(tau_4) # Second-order reduction of the velocity in the z direction
# No és convenient posar directament dx(u) a l'equació ja que faltaria el lift... s'ha de definir bé la derivada
# No definim derivada temporal ja que és un EVP!!!

# RESOLDRE PRIMER SUBSTITUINT Bx, Bx i RHO i només resolent 2 eqs... i 4 eqs de contorn, només resolem per u i w, eqs. (17') i (19')

# velocitat dedins menor que defora
# alfven major primer, després provar

# comparar amb kz = 0 les dues equacions han de ser ~ iguals
# provar amb kz = 0.01 i s'ha de modificar poc - petit acoblament de les velocitats



""" Problem """
### Problema d'ones ràpida i lenta (complet) ###
#problem = d3.EVP([u, w, Bx, Bz, rho0, rho10, tau_1, tau_2,], eigenvalue=s, namespace=locals())
#problem.add_equation("np.sqrt(s)*rho10 + rho0*(-wx - kz*w) =0 ") # (12)
#problem.add_equation("np.sqrt(s)*Bx = kz*B0*w") # (14)
#problem.add_equation("np.sqrt(s)*Bz = B0*ux") # (16)
#problem.add_equation("np.sqrt(s)*rho10*ux = -c2*rho10x ") # (17)
#problem.add_equation("np.sqrt(s)*rho10*w - c2*kz*rho10 - vA2*(rho/B0)*(kz*Bx -Bzx) = 0") # (19)

### Problema d'ones d'Alfvén ### ~ corda guitarra amb densitat variable
#problem = d3.EVP([u, tau_1, tau_2], eigenvalue=s, namespace=locals())
#problem.add_equation("s * u  +  vA2 * uxx  =  0") # (12)

### Problema d'ones ràpida i lenta (més treballat) ###
problem = d3.EVP([u, w, tau_1, tau_2, tau_3, tau_4], eigenvalue=s, namespace=locals()) # 2 equacions i 4 incògnites
# problem.add_equation("s * u  +  c2 * uxx  =  0") # (17') # amb kz = 0
# problem.add_equation("s * w  +  vA2 * wxx  =  0") # (19') # amb kz = 0. Hauríem d'obtenir una molt semblant a la (17') però amb la velocitat en z i la constant d'Alfvén
problem.add_equation("s * u  +  c2 * uxx  +  kz * c2 * wx  =  0") # (17')
problem.add_equation("vA2 * wxx  +  c2 * kz * ux  +  (-c2 * kz**2 - vA2 * kz**2 + s) * w  =  0") # (19')
problem.add_equation("u(x=xmin) = 0")
problem.add_equation("u(x=xmax) = 0")
problem.add_equation("w(x=xmin) = 0")
problem.add_equation("w(x=xmax) = 0")
print("Number of unknowns:", len(problem.variables))
print("Number of equations:", len(problem.equations))



""" Solve """
### Solver de Dedalus ###
solver = problem.build_solver()
#print(solver.subproblems)
solver.solve_dense(solver.subproblems[0])
evals = np.sort(solver.eigenvalues)

### Ajustament de les autofuncions ###
print('evals.size (before) ', evals.size)
#evals = evals[~np.isinf(evals)] # el ~ és un operador bitwise NOT, que retorna True si el valor és False i viceversa. En aquest cas elimina els valors infinits
print('evals.size (after removing inf) ', evals.size)
evals = evals[evals > 0] # eliminam els valors negatius
print('evals.size (after removing < 0) ', evals.size)
evals = np.sqrt(evals)

### Autovalors exactes ###
Nevals = evals.size
exact_soln = get_eigenvalues(Nevals)
n = 1 + np.arange(evals.size)
true_evals = exact_soln[0:evals.size]

### Error relatiu ###
relative_error = np.abs(evals - true_evals) / true_evals

### Autovalors ###
print('evals size', evals.size)
print("true_evals size ", true_evals.size)
#print("Evals", evals)
#print("True Evals", true_evals)

# Agafam la part real per fer un print més net
evals = evals.real
print("First 5 Eigenvalues (sqrt):", evals[:5])
print("First 5 True eigenvalues:", true_evals[:5])
print("Relative error:", relative_error[:5])



""" Plot """
# Plot error
plt.figure(figsize=(6, 4))
plt.semilogy(n, relative_error, '.')
plt.xlabel("Eigenvalue number")
plt.ylabel("Relative eigenvalue error")
plt.tight_layout()
plt.title("Eigenvalue error")
#plt.savefig("eigenvalue_error.pdf")
#plt.savefig("eigenvalue_error.png", dpi=200)


# Plot real part of v10x
plt.figure(figsize=(6, 4))
x = dist.local_grid(xbasis) # no acab d'entendre perquè es repeteix el domini... per si l'has tuneat penmig maybe
for n, idx in enumerate(np.argsort(solver.eigenvalues)[:8], start=1):
    solver.set_state(idx, solver.subsystems[0])
    #ug = u['g'].real
    #if np.max(np.abs(ug)) != 0:  # Avoid division by zero
    #    ug /= np.max(np.abs(ug))
    #plt.plot(x, ug, label=f"n={n}")

    ug = (u['g']).real / (u['g'][1]).real
    #print(u['g'][1])
    plt.plot(x, ug/np.max(np.abs(ug)), label=f"n={n}") # quan es normalitza es divideix per zero... pq hi ha una autofunció nul·la?
plt.xlim(-1, 1)
plt.legend(loc="lower right", frameon=True)
plt.ylabel(r"Mode structure")
plt.xlabel(r"x")
plt.title(r"Real part of $\tilde{v}_{10x}$") # idò és imaginària ---> desfassament temporal de 1/4 de període
plt.tight_layout()
plt.show()
# plt.savefig("real_eigenvectors_v10x~.pdf")
# plt.savefig("real_eigenvectors_v10x~.png", dpi=200)

# Plot imaginary part of v10x
plt.figure(figsize=(6, 4))
x = dist.local_grid(xbasis)
for n, idx in enumerate(np.argsort(solver.eigenvalues)[:8], start=1):
    solver.set_state(idx, solver.subsystems[0])
    # if u['g'][1] != 0:
        # ui = (u['g'] / u['g'][1]).imag
    # else:
        # ui = u['g'].imag  # Skip normalization
    #ug = u['g'].real
    #ui = u['g'].imag
    #if np.max(np.abs(ug)) != 0:  # Avoid division by zero
     #   ui /= np.max(np.abs(ug)) # Normalitzam per la part real (ug), que és molt major i no ens resulta en la mateixa autofunció

    #plt.plot(x, ui, label=f"n={n}")

    ug = (u['g'] / u['g'][1]).real
    ui = (u['g'] / u['g'][1]).imag
    plt.plot(x, ui/np.max(np.abs(ug)), label=f"n={n}") # no normalitzam pq ens divideix per 0 ### ara sí normalitzam
plt.xlim(-1, 1)
plt.legend(loc="lower right", frameon=True)
plt.ylabel(r"Mode structure")
plt.xlabel(r"x")
plt.title(r"Imaginary part of $\tilde{v}_{10x}$") # idò és real ---> desfassament temporal de 1/4 de període
plt.tight_layout()
plt.show()
# plt.savefig("imag_eigenvectors_v10x~.pdf")
# plt.savefig("imag_eigenvectors_v10x~.png", dpi=200)


# Plot real part of v10z
plt.figure(figsize=(6, 4))
x = dist.local_grid(xbasis)
for n, idx in enumerate(np.argsort(solver.eigenvalues)[:8], start=1):
    solver.set_state(idx, solver.subsystems[0])
    #wg = w['g'].real
    #if np.max(np.abs(wg)) != 0:  # Avoid division by zero
    #    wg /= np.max(np.abs(wg))
    #plt.plot(x, wg, label=f"n={n}")

    wg = (w['g'] / w['g'][1]).real
    plt.plot(x, wg/np.max(np.abs(wg)), label=f"n={n}")
plt.xlim(-1, 1)
plt.legend(loc="lower right", frameon=True)
plt.ylabel(r"Mode structure")
plt.xlabel(r"z")
plt.title(r"Real part of ${v}_{10z}$")
plt.tight_layout()
plt.show()
# plt.savefig("real_eigenvectors_v10z.pdf")
# plt.savefig("real_eigenvectors_v10z.png", dpi=200)

# Plot real part of v10z
plt.figure(figsize=(6, 4))
x = dist.local_grid(xbasis)
for n, idx in enumerate(np.argsort(solver.eigenvalues)[:8], start=1):
    solver.set_state(idx, solver.subsystems[0])
    #wi = u['g'].imag
    #if np.max(np.abs(wi)) != 0:  # Avoid division by zero
    #    wi /= np.max(np.abs(wi)) # Normalitzam per la part real (wg), que és molt major i no ens resulta en la mateixa autofunció
    #plt.plot(x, wi, label=f"n={n}")

    wg = (w['g'] / w['g'][1]).real
    wi = (w['g'] / w['g'][1]).imag
    plt.plot(x, wi/np.max(np.abs(wg)), label=f"n={n}")
plt.xlim(-1, 1)
plt.legend(loc="lower right", frameon=True)
plt.ylabel(r"Mode structure")
plt.xlabel(r"z")
plt.title(r"Imaginary part of ${v}_{10z}$")
plt.tight_layout()
plt.show()
# plt.savefig("imag_eigenvectors_v10z.pdf")
# plt.savefig("imag_eigenvectors_v10z.png", dpi=200)
""" """

# A l'hora de representar la part real de les autofuncions has de tenir clar que la velocitat u és en la direcció x i la w en z. També que v10x és imaginària i v10z és real.
# A més, el que es representa és la velocitat, no la corda en sí. Llavors un node implica velocitat nul·la: aquell punt no es mou.
# Els màxims i mínims són els punts on la velocitat és màxima, no és que la corda es deformi amb aquella forma.


Nkz = 50 # Nombre de kz's
kz_list = np.linspace(0, 5, Nkz) # Hi ha Nkz's separats equidistantment entre 0 i 5
### Relació de dispersió ###
plt.figure(figsize=(6, 4))
evals_k = np.zeros((kz_list.size, evals.size), dtype=np.complex128)
for ik, kz in enumerate(kz_list):
    problem = d3.EVP([u, w, tau_1, tau_2, tau_3, tau_4], eigenvalue=s, namespace=locals())
    problem.add_equation("s * u  +  c2 * uxx  +  kz * c2 * wx  =  0") # (17')
    problem.add_equation("vA2 * wxx  +  c2 * kz * ux  +  (-c2 * kz**2 - vA2 * kz**2 + s) * w  =  0") # (19')
    problem.add_equation("u(x=xmin) = 0")
    problem.add_equation("u(x=xmax) = 0")
    problem.add_equation("w(x=xmin) = 0")
    problem.add_equation("w(x=xmax) = 0")

    solver = problem.build_solver()
    solver.solve_dense(solver.subproblems[0])

    evals_k[ik, :] = np.sort(solver.eigenvalues)

# Plot dispersion relation of the first 8 eigenvalues
for i in range(8):
    colors = ['red', 'green', 'blue', 'orange', 'purple', 'brown', 'pink', 'gray']
    plt.plot(kz_list, np.sqrt(evals_k[:, i].real), '.', label=f"n={i+1}", color=colors[i % len(colors)])
plt.xlabel(r"$k_z$")
plt.ylabel(r"$\omega = \sqrt{s}$")
plt.title("Dispersion Relation")
plt.grid(True)    
plt.legend(loc="upper right", frameon=True, fontsize=8)
plt.show()

# Per veure amb més detall la relació de dispersió de cada mode la representam individualment.
for i in range(8):
    colors = ['red', 'green', 'blue', 'orange', 'purple', 'brown', 'pink', 'gray']
    plt.plot(kz_list, np.sqrt(evals_k[:, i].real), '.', label=f"Mode {i+1}", color=colors[i % len(colors)])
    plt.xlabel(r"$k_z$")
    plt.ylabel(r"$\omega = \sqrt{s}$")
    plt.title(f"Dispersion Relation for n = {i+1}")
    plt.grid(True)
    plt.show()
plt.show()

# Tema apart: estic emprant ipynb per tant tots els triples # podrien ser entrades ò cel·les, no fa falta fer-ho tot en un. Per això ja hi ha es .py

p0  0.004131736526946108
rho0c  2.5e-13
rho0c  2.5e-13
csp, csc, vAp, vAc  11735.610376039398 165966.59356521335 63078.313050504 892062.0580763856
cp2, cc2, vAp2, vAc2  137724550.89820358 27544910179.640717 3978873577.297384 795774715459.4768
Number of unknowns: 6
Number of equations: 6
2025-04-25 13:10:32,621 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 20s, Remaining: 0s, Rate: 5.0e-02/s
evals.size (before)  260
evals.size (after removing inf)  260
evals.size (after removing < 0)  249


KeyboardInterrupt: 